In [47]:
# !pip install --upgrade openai

In [ ]:
from openai import OpenAI

# create an OpenAI client using the API key
client = OpenAI(api_key=API_KEY)

In [ ]:
# import pymupdf4llm
# md_text = pymupdf4llm.to_markdown("6832_SercePehlevan_2020.pdf")

# import pathlib
# pathlib.Path("output.md").write_bytes(md_text.encode())

In [35]:
file = client.files.create(
    file=open("6832_SercePehlevan_2020.pdf", "rb"),
    purpose="user_data",
)

In [44]:
def generate_response_with_chatgpt(prompt):
    response = client.responses.create(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": [
                    { "type": "input_text", "text": prompt },
                    {
                        "type": "input_file",
                        "file_id": file.id,
                    }
                ]
            }
        ]
    )

    return response.output_text

In [ ]:
with open("domain_1_q_1_1.txt", "r", encoding="utf-8") as f:
    content = f.read()

answer = generate_response_with_chatgpt(content)

In [ ]:
from pathlib import Path

def process_domain_files(patterns=("domain_1*.txt", "domain_2*.txt")):
    """Read each domain text file and send its content to the model."""
    answers = {}
    for pattern in patterns:
        for path in sorted(Path('.').glob(pattern)):
            with path.open('r', encoding='utf-8') as f:
                content = f.read()
            response = generate_response_with_chatgpt(content)
            answers[path.name] = getattr(response, 'output_text', response)
    return answers

all_answers = process_domain_files()
for name, answer in all_answers.items():
    print(f"{name} -> {answer}")
